In [2]:
import requests
from bs4 import BeautifulSoup 
from tqdm import tqdm
import pandas as pd

In [3]:
banks = ["akbank","halkbank","is%20bankasi","yapi%20kredi","vakifbank","garanti","denizbank"]
web_links = {
    "borsa_gundem":"https://www.borsagundem.com/ara/{bank}/{i}",
    "bloomberght":"https://www.bloomberght.com/liste/arama/{i}?ara={bank}&baslangic_tarihi=&bitis_tarihi=",
    "hurriyet":"http://bigpara.hurriyet.com.tr/arama/?firstDate=2015-06-01&secondDate=2020-09-28&includeKap=False&page={i}&s={bank}",
    "finans_gundem":"https://www.finansgundem.com/ara/{bank}/{i}"
    
}

In [4]:
def borsa_gundem(array,bank):
    for i in tqdm(range(1,75)):
        page = requests.get(web_links["borsa_gundem"].format(bank=bank,i=i))
        sp = BeautifulSoup(page.content,"html.parser")
        liste = sp.find("div",{"class":"srch"}).find_all("li")
        for l in liste: 
            try:
                title = l.strong.text
                tarih = l.small.text
                tarih = tarih[:tarih.find(",")]
                content = ""
                pg = requests.get(l.a.get("href"))
                psp = BeautifulSoup(pg.content,"html.parser")
                for i in list(psp.find("div",{"class":"dtail"}).find_all("p")[:-2]):
                    content += i.text.replace("\xa0"," ")
                if not content:
                    continue
                array.append([title,content,tarih,bank])
            except Exception as e:
                print(l.a.get("href"),"  ",e)
                continue    

def bloomberght(array,bank):
    adres = "https://www.bloomberght.com"
    for i in tqdm(range(1,50)):
        page = requests.get(web_links["bloomberght"].format(bank=bank,i=i))
        sp = BeautifulSoup(page.content,"html.parser")
        liste = [adres + i.a.get("href") for i in sp.find_all("div",{"class":"box box-12 item"}) if not "/video/" in i.a.get("href")]

        for l in liste:
            try:
                pg = requests.get(l)
                psp = BeautifulSoup(pg.content,"html.parser")

                title = psp.h1.span.text
                tarih = psp.find("div",{"class":"news-info"}).span.time.text
                tarih = " ".join(tarih.split()[:3])
                content =""
                for i in list(psp.article.find_all("p")):
                    content += i.text
                array.append([title,content,tarih,bank])
            except Exception as e:
                print(l, "  ", e)
                
def hurriyet(array,bank):
    adres = "http://bigpara.hurriyet.com.tr"
    for i in tqdm(range(1,30)):   
        page = requests.get(web_links["hurriyet"].format(bank=bank,i=i))
        sp = BeautifulSoup(page.content, "html.parser")
        liste = sp.find("div",{"class":"aramasonuc"}).find_all("li")
        
        for l in liste:
            try:
                title = l.a.text
                icerik = l.find_all("a")[1].text
                tarih = l.span.text[1:-1]
                link = adres + l.a.get("href")
                pg = requests.get(link)
                psp = BeautifulSoup(pg.content, "html.parser")
                content = ""
                for i in list(psp.find("div",{"class","tag-content"}).find_all("p")):
                    content += i.text.replace("\xa0"," ")

                array.append([title,content,tarih,bank])
            except:
                print(link)
                continue


def finans_gundem(array,bank):
    for i in tqdm(range(1,75)):
        page = requests.get(web_links["finans_gundem"].format(bank=bank,i=i))
        sp = BeautifulSoup(page.content,"html.parser")
        liste = sp.find("div",{"class":"srch"}).find_all("li")
        for l in liste: 
            try:
                title = l.strong.text
                tarih = l.small.text
                tarih = tarih[:tarih.find(",")]
                content = ""
                pg = requests.get(l.a.get("href"))
                psp = BeautifulSoup(pg.content,"html.parser")
                for i in list(psp.find("div",{"class":"dtail"}).find_all("p")[:-2]):
                    content += i.text.replace("\xa0"," ")
                if not content:
                    continue
                array.append([title,content,tarih,bank])
            except Exception as e:
                print(l.a.get("href"),"  ",e)
                continue    



In [5]:
column_names = ["title","content","date","bank"]

array = []
for bank in banks:
    borsa_gundem(array,bank)
    bloomberght(array,bank)    
    hurriyet(array,bank)    
    finans_gundem(array,bank)
    data = pd.DataFrame(data=array,columns=column_names)
    data.to_csv("../Data/banks.csv",index=False)

 11%|████████▉                                                                          | 8/74 [02:27<21:44, 19.77s/it]

https://www.borsagundem.com/redirect/1349928    'NoneType' object has no attribute 'find_all'


 57%|██████████████████████████████████████████████▌                                   | 42/74 [13:16<09:17, 17.43s/it]

https://www.borsagundem.com/redirect/1051036    'NoneType' object has no attribute 'find_all'


 58%|███████████████████████████████████████████████▋                                  | 43/74 [13:33<08:56, 17.32s/it]

...

  5%|████▍                                                                              | 4/74 [01:02<18:08, 15.55s/it]

https://www.borsagundem.com/redirect/1351398    'NoneType' object has no attribute 'find_all'


 12%|█████████▎                                                                   | 9/74 [1:26:22<10:23:52, 575.88s/it]

https://www.borsagundem.com/haber/milyonlari-ilgilendiren-tasari-yasalasti/1304879    ('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))
https://www.borsagundem.com/haber/penta-teknoloji-halka-arzla-birlikte-yatirimlarini-artiracak/1304046    HTTPSConnectionPool(host='www.borsagundem.com', port=443): Max retries exceeded with url: /haber/penta-teknoloji-halka-arzla-birlikte-yatirimlarini-artiracak/1304046 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000245C90FF4C8>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
https://www.borsagundem.com/redirect/1304051    HTTPSConnectionPool(host='www.borsagundem.com', port=443): Max retries exceeded with url: /redirect/1304051 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000245C90FFB08>: Failed to establish a new connection: [Errno 110

ConnectionError: HTTPSConnectionPool(host='www.borsagundem.com', port=443): Max retries exceeded with url: /ara/yapi%20kredi/10 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000245C90FF688>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))